In [1]:
"""
Projekt: Czynniki wplywajace na wystepowanie choroby serca

Autor: Zuzanna Kontna

Studia Podyplomowe: Data Scientist. Analityk danych
Uniwersytet WSB Merito Gdańsk
kwiecien, 2023
"""

'\nProjekt: Czynniki wplywajace na wystepowanie choroby serca\n\nAutor: Zuzanna Kontna\n\nStudia Podyplomowe: Data Scientist. Analityk danych\nUniwersytet WSB Merito Gdańsk\nkwiecien, 2023\n'

In [2]:
# import bibliotek
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns
import plotly.io as pio

In [3]:
# otwieranie wykresow z plotly w przegladarce
pio.renderers.default='browser'

In [4]:
# zrodlo danych: https://www.kaggle.com/datasets/sulianova/cardiovascular-disease-dataset

# definicje poszczegolnych kolumn:
    # ID: unikatowe id obserwacji
    # Age: wiek pacjenta (wyrazony w dniach)
    # Gender: plec pacjenta (1 - kobieta, 2 - mezczyzna)
    # Height: wzrost pacjenta (w cm)
    # Weight: masa pacjenta (w kg)
    # Ap_hi: skurczowe cisnienie krwi
    # Ap_lo: rozkurczowe cisnienie krwi
    # Cholesterol: cholesterol (1 - w normie, 2 - powyzej normy, 3 - znacznie powyzej normy)
    # Glucose: glukoza (1 - w normie, 2 - powyzej normy, 3 - znacznie powyzej normy)
    # Smoke: oznaczenie, czy pacjent pali papierosy (0 - nie pali, 1 -pali)
    # Alco: oznaczenie, czy pacjent pije alkohol (0 - nie pije, 1 - pije)
    # Active: oznaczenie, czy pacjent jest aktywny fizycznie (0 - nie jest, 1 - jest)
    # Cardio: wystepowanie choroby serca (0 - nie wystepuje, 1 - wystepuje)

In [5]:
# ustawienie katalogu roboczego
os.chdir('C:\\Users\\zuza2\\OneDrive\\Pulpit\\podyplomowka\\projekt')

In [6]:
# import danych pliku
cardio = pd.read_csv('cardio.csv', sep = ';')

In [7]:
# podglad na pierwszych 10 wierszy 
cardio_top_10 = cardio.head(10)
print(cardio_top_10)

   id    age  gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  smoke  \
0   0  18393       2     168    62.0    110     80            1     1      0   
1   1  20228       1     156    85.0    140     90            3     1      0   
2   2  18857       1     165    64.0    130     70            3     1      0   
3   3  17623       2     169    82.0    150    100            1     1      0   
4   4  17474       1     156    56.0    100     60            1     1      0   
5   8  21914       1     151    67.0    120     80            2     2      0   
6   9  22113       1     157    93.0    130     80            3     1      0   
7  12  22584       2     178    95.0    130     90            3     3      0   
8  13  17668       1     158    71.0    110     70            1     1      0   
9  14  19834       1     164    68.0    110     60            1     1      0   

   alco  active  cardio  
0     0       1       0  
1     0       1       1  
2     0       0       1  
3     0       1

In [9]:
# dodaje nowa kolumne z wiekiem wyrazonym w latach, zaokraglonym w dol, czyli do ukonczonego wieku
cardio['age_years'] = cardio['age'].apply(lambda a: a//365)

In [10]:
# podsumowanie kazdej z kolumn
cardio_desc = cardio.describe()
cardio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
 13  age_years    70000 non-null  int64  
dtypes: float64(1), int64(13)
memory usage: 7.5 MB


In [11]:
# brak nulli
cardio.isnull().sum()

id             0
age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
age_years      0
dtype: int64

In [12]:
# id nie bedzie mi potrzebne, usuwam je
cardio.drop(columns=['id'], inplace=True)
cardio = cardio.reset_index(drop = True)

In [13]:
# ocena poprawnosci danych

In [14]:
# cisnienie
# z google: The highest pressure recorded in an individual was 370/360. Zatem usuwam wszystko, co powyżej. Usuwam też cisnienie nizsze od 0.

cardio.drop(cardio[cardio['ap_hi'] >= 360].index, inplace = True)
cardio.drop(cardio[cardio['ap_hi'] < 0].index, inplace = True)
cardio.drop(cardio[cardio['ap_lo'] >= 370].index, inplace = True)
cardio.drop(cardio[cardio['ap_lo'] < 0].index, inplace = True)

In [15]:
### wizualizacja danych

In [16]:
# cisnienie krwi

ap_fig = make_subplots(rows=1, cols=2)

ap_fig.add_trace(
    go.Box(y = cardio['ap_hi'],
           name = 'Ap_hi'
               ),
    row = 1,
    col = 1
    )
              
ap_fig.add_trace(
    go.Box(y = cardio['ap_lo'],
           name = 'Ap_lo'
           ),
    row = 1,
    col = 2       
    )

In [17]:
# wzrost i masa

# doliczam BMI
cardio['BMI'] = cardio['weight']/((cardio['height']/100)*(cardio['height']/100))

In [18]:
# wizualizacja danych

dim_fig = make_subplots(rows=1, cols=3)

dim_fig.add_trace(
    go.Box(y = cardio['weight'],
           name = 'Weight'
           ),
    row = 1,
    col = 1
    )

dim_fig.add_trace(
    go.Box(y = cardio['height'],
           name = 'Height'
           ),
    row = 1,
    col = 2
    )

dim_fig.add_trace(
    go.Box(y = cardio['BMI'],
           name = 'BMI'
           ),
    row = 1,
    col = 3
    )

In [19]:
# z google: the lowest reported BMI was only 6.7, odrzucam zatem rekordy z BMI < 6.7
cardio.drop(cardio[cardio['BMI'] < 6.7].index, inplace = True)

In [31]:
# wzrost wyglada podejrzanie, przygladam sie dokladniej

height_fig = px.histogram(
    data_frame = cardio,
    x = 'height',
    color = 'gender'
    )

# zmieniam etykiety danych na wykresie
labels = {'1': 'female', '2': 'male'}

height_fig.for_each_trace(lambda t: t.update(name = labels[t.name],
                                             legendgroup = labels[t.name],
                                             hovertemplate = t.hovertemplate.replace(t.name, labels[t.name])
                                             )
                          )

height_fig.show()

In [21]:
# mam podejrzenie, ze tutaj rowniez sa nierealne dane, weryfikuje to z wiekiem

In [22]:
# wiek - wizualizacja danych

age_fig = px.histogram(
    data_frame = cardio,
    x = 'age_years'
    )

age_fig.show()

In [23]:
# poniewaz w zbiorze nie ma dzieci, odcinam rekordy, dla ktorych wzrost jest nizszy niz 140 oraz wyzszy niz 220
cardio.drop(cardio[cardio['height'] < 140].index, inplace = True)
cardio.drop(cardio[cardio['height'] > 220].index, inplace = True)

In [24]:
# w tym miejscu koncze usuwanie 'pojejrzanych danych'

In [25]:
# tworze kopie zbioru cardio i zamieniam zmienne kategoryzujace gender, cholesterol, glucose, smoke, alcohol, active, cardio) na opisy

cardio_1 = cardio.copy(deep = False)

cardio_1['gender'] = cardio_1['gender'].replace(to_replace = [1, 2], value = ['female', 'male'])
cardio_1['cholesterol'] = cardio_1['cholesterol'].replace(to_replace = [1, 2, 3], value = ['normal', 'above normal', 'well above normal'])
cardio_1['gluc'] = cardio_1['gluc'].replace(to_replace = [1, 2, 3], value = ['normal', 'above normal', 'well above normal'])
cardio_1['smoke'] = cardio_1['smoke'].replace(to_replace = [0, 1], value = ['no', 'yes'])
cardio_1['alco'] = cardio_1['alco'].replace(to_replace = [0, 1], value = ['no', 'yes'])
cardio_1['active'] = cardio_1['active'].replace(to_replace = [0, 1], value = ['no', 'yes'])
cardio_1['cardio'] = cardio_1['cardio'].replace(to_replace = [0, 1], value = ['no', 'yes'])

In [26]:
# wizualizuje zmienne kategoryzujace i czestosc ich wystepowania (juz na oczyszczonym zbiorze): gender, cholesterol, glucose, smoke, alcohol, active
vars = ['gender', 'cholesterol', 'gluc', 'smoke', 'alco', 'active']

colors = {'no': 'green', 'yes': 'red'}

cat_fig = make_subplots(rows = 2, cols = 3, subplot_titles = ('Gender', 'Cholesterol', 'Glucose', 'Smoke', 'Alcohol', 'Active'))

for i, var in enumerate(vars):
    row = i//3 + 1
    col = i % 3 +1
    for card in ['no', 'yes']:
        trace = go.Histogram(x = cardio_1[cardio_1['cardio'] == card][var],
                             name = card,
                             marker_color = colors[card],
                             opacity = 0.8                             
                             )
        cat_fig.add_trace(trace, row = row, col = col)

cat_fig.update_layout(title = 'Categories by cardiovascular disease')

cat_fig.show()

In [27]:
# ostatnia z wizualizacji pomagajacych zapoznac sie ze zbiorem - korelacje pomiedzy zmiennymi (kazda z kazda)
heatmap_data = cardio.copy(deep = False)
heatmap_data = heatmap_data.filter(['age_years', 'gender', 'height', 'weight', 'BMI', 'ap_hi', 'ap_lo', 'cholesterol', 
                                    'gluc', 'smoke', 'alco', 'active', 'cardio'])


heat_fig = px.imshow(heatmap_data.corr(),
                     aspect = 'auto',
                     text_auto = True #warosci liczbowe na wykresie
                     )
heat_fig.show()

In [27]:
# eksportuje pliki cardio i cardio_1, aby w R Studio zbudowac do nich modele machine learningowe
cardio.to_csv('cardio_exit.csv', index = False)

cardio_1.to_csv('cardio_1_exit.csv', index = False)